 #### Refaire l'experience en utilisant des petits $B_\theta$ (<15°)

In [1]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random
from psychopy import visual, core, event, gui, data
import MotionClouds as MC
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
exp_name = "Perception de l'orientation"
exp_info = {'participant':''}
subj_info = gui.DlgFromDict(dictionary=exp_info, title=exp_name)
exp_info['exp_name'] = exp_name
data_folder = os.path.join('..', 'Psychopy_data', 'pilot', 'mouse_motion','json')
data_folder

'../Psychopy_data/pilot/mouse_motion/json'

In [3]:
%mkdir -p {data_folder}

In [4]:
downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

In [5]:
test_length = 150     # number of trials
fixation_length = .5  # length of fixation, in second
stim1_length = 0.25   # length of first stim, in second
stim2_length = 0.25   # length of the second stim, in second 

fullscr = False

fullscr = True


In [6]:
def generate_random_cloud(theta, B_theta, B_sf, downscale = 1):
    fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = 'tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

In [7]:
%mkdir -p tmp

In [8]:
#win = visual.Window([1400,800],fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])


mouse = event.Mouse(newPos=(0, 0), visible=False)
x0, y0 = mouse.getPos()
event.clearEvents()
instructions = """
At each trial you see one textured image, report the 
orientation you perceive.

Move the mouse to the right if it is clock-wise of the vertical,
move the mouse to the left if it is counter clock-wise of the vertical.

Move the mouse to begin the experiment.
"""
msg = visual.TextStim(win, text=instructions, alignHoriz='center', alignVert='center', color='black')
msg.draw()
win.flip()
while not mouse.mouseMoved(distance=.05): 
    event.clearEvents()
    
fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2, B_sf=0.1)
bitmap1 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap1.autolog = False 

msg = visual.TextStim(win, text='?', color='black')

import time
start_time = time.time()
print("Test started")


ans_list = []
N_B_thetas = 5 
N_B_Bsf = 5
B_thetas = np.pi*np.logspace(-7, -3.5, N_B_thetas, base=2)
B_sfs = np.logspace(-4,0.,N_B_Bsf,base=2)
    
std_theta = np.pi/6
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()
    
    theta = std_theta * np.random.randn()
    
    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]
    B_sf = B_sfs[random.randint(0, N_B_Bsf-1)]
    MC1 = generate_random_cloud(np.pi/2-theta, B_theta=B_theta, B_sf=B_sf) 
    bitmap1.setImage(MC1)

    # Times the trial
    while clock.getTime() < fixation_length + stim1_length + stim2_length:
        if 0.0 <= clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length: 
            bitmap1.draw()
            win.flip()
        if fixation_length + stim1_length <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            msg.draw()
            
    win.flip()

    mouse.setPos(newPos=(0, 0))
    x0, y0 = mouse.getPos()
    event.clearEvents()

    while not mouse.mouseMoved(distance=.05): 
        # this creates a never-ending loop
        # until we move the mouse
        event.clearEvents()

    x, y = mouse.getPos()
    if x < 0:
        answer = 'left'
    else: #elif x > 0:
        answer = 'right'
        if x == 0: print('this should not happen 😤')
    event.clearEvents() 
    # print('answer', answer, ', x', x)

    ans = event.getKeys()   

    if ans == ['escape', 'q']:
        win.close()
        core.quit()

    correct = ((theta > 0) and answer == 'right') or ((theta < 0) and answer == 'left')
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ',  answer, '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi),'B_sf=', B_sf)   
       
    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, answer, B_theta, B_sf])

win.close()

Test started
At trial  0 Angle= 31.915 answer is  right (correct= True ); bandwidth= 2.579 B_sf= 0.125
At trial  1 Angle= 0.392 answer is  left (correct= False ); bandwidth= 8.675 B_sf= 1.0
At trial  2 Angle= -59.607 answer is  left (correct= True ); bandwidth= 1.406 B_sf= 0.5
At trial  3 Angle= -14.853 answer is  left (correct= True ); bandwidth= 4.730 B_sf= 1.0
At trial  4 Angle= 38.416 answer is  right (correct= True ); bandwidth= 4.730 B_sf= 0.0625
At trial  5 Angle= 33.060 answer is  right (correct= True ); bandwidth= 1.406 B_sf= 0.5
At trial  6 Angle= 7.635 answer is  right (correct= True ); bandwidth= 8.675 B_sf= 0.125
At trial  7 Angle= 39.933 answer is  right (correct= True ); bandwidth= 8.675 B_sf= 0.5
At trial  8 Angle= 4.841 answer is  right (correct= True ); bandwidth= 15.910 B_sf= 1.0
At trial  9 Angle= -19.118 answer is  left (correct= True ); bandwidth= 8.675 B_sf= 1.0
At trial  10 Angle= -5.835 answer is  left (correct= True ); bandwidth= 1.406 B_sf= 0.125
At trial  11

In [10]:
import json
json.dump(ans_list, open( os.path.join(data_folder, 'Psychophys_discrim_%s.json' % exp_info['participant']), 'w', encoding="utf8"))
